In [2]:
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets

In [3]:
from torch.utils.tensorboard import SummaryWriter

In [4]:
# custom libraries
root_main = os.getcwd()
os.chdir("..")
import TorchCommon as TC
os.chdir(root_main)

In [5]:
# main pyperparametrs
valid_size=0.2
nrm_mean=0.5
nrm_std=0.5
num_workers=0
root_ds = "D:\GitHub\pytorch-lab\Dataset" # dataset root
root_bm = "D:\GitHub\pytorch-lab\Best_Models" # best models root

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# transform
trans=transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((nrm_mean,) , (nrm_std,))
])

In [14]:
#load dataset
train_data=datasets.MNIST(root=root_ds,
                          train=True, transform=trans, download=True)

In [15]:
#sampler
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)

In [46]:
# Network
class Net(nn.Module):
    # Assign Intrinsic Properties of Your Neural Network
    def __init__(self):
        super().__init__()
        # Weights of Layer 1th and Layer 2th Are Intrinsic Properties
        self.fc1 = nn.Linear(784, 256, bias=True)
        self.fc2 = nn.Linear(256, 100, bias=True)
        self.fc3 = nn.Linear(100, 10, bias=True)

    # Wiring of Your Network
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = F.relu_(x)
        x = self.fc2(x)
        x = F.relu_(x)
        x = self.fc3(x)
        return x

# TensorBoard

### Creating a loss and accuracy plot (independently)

In [17]:
# Hyperparaameter
epochs=5
batch_size=64
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_scalar("Loss Train", scalar_value= loss, global_step=step)
        writer.add_scalar("Accuracy Train", scalar_value= acc, global_step=step)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### Creating a loss and accuracy plot (in same plot)

In [37]:
# Hyperparaameter
epochs=5
batch_size=64
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_scalars("Loss/Accuracy", {'Loss' : loss, 'Accuracy': acc}, global_step=step)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### Histogram

In [50]:
# Hyperparaameter
epochs=5
batch_size=64
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_histogram("FC1_Weights", model.fc1.weight, global_step=step)
        writer.add_histogram("FC2_Weights", model.fc2.weight, global_step=step)
        writer.add_histogram("FC3_Weights", model.fc3.weight, global_step=step)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### Graph

In [48]:
# Hyperparaameter
batch_size=64

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net()

# TesnorBoard File
root_tb=f'runs/batch {batch_size}'
writer = SummaryWriter(root_tb)

# -NEW- TensorBoard process
image,label=next(iter(train_loader))
writer.add_graph(model, image)
# -NEW- TensorBoard process
writer.close()

### mini-batch image

In [65]:
# Hyperparaameter
epochs=1
batch_size=61
lr=0.1

# TrainLoader
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

# Loss and Optimizer
model=Net().to(device)
criterion =nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(), lr=lr)

# TesnorBoard File
root_tb=f'runs/batch {batch_size} LR {lr}'
writer = SummaryWriter(root_tb)

# Train
step=0
for epoch in range(1,epochs+1):
    for iter_train, (image, label) in enumerate(train_loader, 1):
        # preprocess and feedforward process
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        y_hat = model(image)
        # we want to track loss/acc
        loss = criterion(y_hat, label)
        acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
        # updating process
        loss.backward()
        optimizer.step()

        # -NEW- TensorBoard process
        writer.add_images("mini=batch data", image, global_step=iter_train)
        step+=1
        # -NEW- TensorBoard process
writer.close()

### mini-batch signal

### Hyperparameters Searching

In [ ]:
# Hyperparaameter
epochs=1
for batch_size in [10,100,1000]:
    for lr in [0.1, 0.01, 0.001]:
        # TrainLoader
        train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)

        # Loss and Optimizer
        model=Net().to(device)
        criterion =nn.CrossEntropyLoss()
        optimizer=optim.SGD(model.parameters(), lr=lr)

        # TesnorBoard File
        root_tb=f'runs/batch {batch_size} LR {lr}'
        writer = SummaryWriter(root_tb)

        # Train
        for epoch in range(1,epochs+1):
            loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)

        # -NEW- TensorBoard process
        writer.add_hparams({'batch_size' : batch_size, 'LR' : lr} ,
                           {'Loss train' : loss_train, 'Accuracy train' : acc_train})
        # -NEW- TensorBoard process
        writer.close()

In [49]:
epochs=1
for batch_size in [2,64,1024]:
    for lr in [0.1, 0.01, 0.001]:
        train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
        #create model and set loss function and optimizer
        model=Net().to(device)
        criterion =nn.CrossEntropyLoss()
        optimizer=optim.SGD(model.parameters(), lr=lr)
        root_tb=f'runs/batch {batch_size} LR {lr}'
        writer = SummaryWriter(root_tb)

        for epoch in range(1,epochs+1):
            loss_train, acc_train = TC.train(model,train_loader, device, optimizer, criterion, epoch)
            #loss_valid, acc_valid = TC.valid(model, valid_loader, device, criterion)
            #loss_valid_min = TC.save_model(model, optimizer, epoch, root_bm, loss_valid_min, loss_valid)

            writer.add_scalar("train loss", scalar_value= loss_train, global_step=epoch)
            writer.add_scalar("train accuracy", scalar_value= acc_train, global_step=epoch)
            writer.add_hparams({'batch_size' : batch_size, 'LR' : lr} , {'Loss' : loss_train, 'Accuracy' : acc_train})

############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 2.25	Acc_Train : 0.14
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.34	Acc_Train : 0.89
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 0.61	Acc_Train : 0.83
############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 1.93	Acc_Train : 0.47
############################################################
###### Epoch 1 #############################################
##########

# Visualizing Dataset Images and Network Weights

In [60]:
epochs=1
#loss_valid_min=np.Inf
step=0
for batch_size in [64]:
    for lr in [0.001]:
        train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
        model=Net().to(device)
        criterion =nn.CrossEntropyLoss()
        optimizer=optim.SGD(model.parameters(), lr=lr)
        root_tb=f'runs/batch {batch_size} LR {lr}'
        writer = SummaryWriter(root_tb)
        for epoch in range(1,epochs+1):
            print(60 * "#")
            print(6 * "#" + " Epoch " + str(epoch) + " " + 45 * "#")
            print(60 * "#")
            loss_train, acc_train = 0, 0
            model.train()
            for iter_train, (image, label) in enumerate(train_loader, 1):
                image, label = image.to(device), label.to(device)

                img_grid=torchvision.utils.make_grid(image)
                writer.add_image("mnist image", img_grid)

                optimizer.zero_grad()
                y_hat = model.(image)
                loss = criterion(y_hat, label)
                acc = np.count_nonzero(label.to('cpu').numpy() == y_hat.argmax(dim=1).to('cpu').numpy())
                loss_train += loss.item() * image.size(0)
                acc_train += acc
                loss.backward()
                optimizer.step()

                writer.add_scalar("Training Loss", loss, global_step=step)
                writer.add_scalar("Training Acc", acc, global_step=step)
                writer.add_histogram("Weights Layer One", model.fc1.weight, global_step=step)
                step+=1

            loss_train /= len(train_loader.sampler)
            acc_train /= len(train_loader.sampler)
            print("Loss_Train: {:.2f}\tAcc_Train : {:.2f}".format(loss_train, acc_train))

############################################################
###### Epoch 1 #############################################
############################################################
Loss_Train: 2.26	Acc_Train : 0.25


In [61]:
len(train_data)

60000

In [90]:
batch_size=200
train_loader=DataLoader(train_data, batch_size= batch_size, num_workers=num_workers, sampler=train_sampler)
writer = SummaryWriter(f"runs")
classes=['0','1','2','3','4','5','6','7','8','9']
for image, label in train_loader:
    x=image
    y=label

In [91]:
features= x.flatten(start_dim=1)
class_labels= [ classes[i] for i in y ]

In [92]:
writer.add_embedding(mat = features,
                 metadata =  class_labels,
                 label_img=x,
                     global_step=1)

# Tensorboard Embedding Projector

In [96]:
# helper function
writer = SummaryWriter(f"runs")
classes=['0','1','2','3','4','5','6','7','8','9']
def select_n_random(data, labels, n=1000):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# select random images and their target indices
images, labels = select_n_random(train_data.data, train_data.targets)

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]

# log embeddings
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.close()

In [2]:
images.shape

NameError: name 'images' is not defined